<a href="https://colab.research.google.com/github/Jaydenzk/DS-Unit-4-Sprint-2-Neural-Networks/blob/master/module4-Hyperparameter-Tuning/LS_DS_424_Hyperparameter_Tuning_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Hyperparameter Tuning

## *Data Science Unit 4 Sprint 2 Assignment 4*

## Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: <https://drive.google.com/file/d/1dfbAsM9DwA7tYhInyflIpZnYs7VT-0AQ/view> 

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


In [2]:
pip install wandb

     |████████████████████████████████| 1.3MB 2.8MB/s 
     |████████████████████████████████| 92kB 28.5MB/s 
     |████████████████████████████████| 235kB 48.8MB/s 
     |████████████████████████████████| 460kB 40.9MB/s 
     |████████████████████████████████| 102kB 33.4MB/s 
     |████████████████████████████████| 92kB 27.4MB/s 
     |████████████████████████████████| 256kB 50.2MB/s 
     |████████████████████████████████| 71kB 15.4MB/s 
     |████████████████████████████████| 184kB 56.6MB/s 
  Created wheel for gql: filename=gql-0.1.0-cp36-none-any.whl size=5541 sha256=0c616110c0b41a059c9b3f0785498767b36343a5cc7b27fd64895edc138e5b7c
  Stored in directory: /root/.cache/pip/wheels/a7/8d/65/a3247f500d675d80a01e4d2f0ee44fe99f1faef575bc2a1664
  Created wheel for watchdog: filename=watchdog-0.9.0-cp36-none-any.whl size=73652 sha256=be1b6d2b288f6a03606134ec7f1de1f1a7d727cb94188497e5710938d13aad5a
  Stored in directory: /root/.cache/pip/wheels/61/1d/d0/04cfe495619be2095eb8d89a31c42adb4e42b7

In [30]:
pip install seed

     |████████████████████████████████| 368kB 2.8MB/s 
  Created wheel for seed: filename=seed-0.11.3-cp36-none-any.whl size=17737 sha256=75e230676831cef4a653972ee7979577de227ac166d9f03e0f30b8f1d7d91812
  Stored in directory: /root/.cache/pip/wheels/8a/95/b2/554d88be6531ab3a370352a919436c2acb785b940bf6221ec3
Successfully built seed


In [0]:
##### Your Code Here #####

import keras
import tensorflow
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
import wandb
import numpy as np
import random
import seed

In [0]:
np.random.seed(42)

In [4]:
!wandb login

wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice: 1
wandb: You chose 'Create a W&B account'
wandb: Create an account here: https://app.wandb.ai/authorize?signup=true
wandb: Paste an API key from your profile and hit enter: bf2a292a009c2f1eaa2a7e86356b6bb9ccda9ee8
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
Successfully logged in to Weights & Biases!


In [0]:
df = pd.read_csv('https://raw.githubusercontent.com/Jaydenzk/DS-Unit-4-Sprint-2-Neural-Networks/master/module4-Hyperparameter-Tuning/WA_Fn-UseC_-Telco-Customer-Churn.csv')

In [0]:
df_encoded = df.copy()

In [22]:
# Split Data into X and y
X = df[['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure',
       'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 
       'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV',
       'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod']]

y = df['Churn']


# Encode Data with LabelEncoder
encoder = LabelEncoder()

X_encoded = X.apply(encoder.fit_transform)
y_encoded = encoder.fit_transform(y)

# Add data that didn't need encoded
X_encoded['MonthlyCharges'] = df['MonthlyCharges']
X_encoded['TotalCharges'] = df['TotalCharges']

# Clean the TotalCharges column to replace missing values
X_encoded['TotalCharges'] = X_encoded['TotalCharges'].replace(' ', '0')
X_encoded['TotalCharges'] = X_encoded['TotalCharges'].astype('float')

# Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X_encoded,
                                                    y_encoded,
                                                    test_size=0.33)



# Create model wrapper
def create_model(n_neurons = 20):
  model = Sequential()

  # Add Input Layer
  model.add(Dense(n_neurons, activation='relu', input_shape=(X_train.shape[1],)))

  # Add Hidden Layers
  model.add(Dense(n_neurons, activation='relu'))
  model.add(Dense(n_neurons, activation='relu'))

  # Add Output Layer
  model.add(Dense(1, activation='sigmoid'))

  # Compile the model
  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
  
  return model

# Create model
model = KerasClassifier(build_fn=create_model, verbose=0)

# Define the grid search parameters
param_grid = {'n_neurons': [15],
              'batch_size': [30]}

# Create Grid Search

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=2, cv=3)
grid_result = grid.fit(X_train, y_train)

print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}") 

# Fit the model
# model.fit(X_train, y_train,
#           validation_data=(X_test, y_test),
#           epochs=wandb.config.epochs,
#           batch_size=wandb.config.batch_size,
#           callbacks=[WandbCallback()])


Best: 0.7132259461524982 using {'batch_size': 30, 'n_neurons': 15}
Means: 0.7132259461524982, Stdev: 0.033157044368083935 with: {'batch_size': 30, 'n_neurons': 15}


## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset using hyperas or hyperopt (if you're brave)
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?